# Molecular Vae implementation with pytorch

※基本的にはhttps://github.com/ShabbirK/chemical_vae_pytorch/blob/main/Chemical%20VAE.ipynbの写経

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm

import pathlib
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## Declare Constants

In [2]:
MAX_LEN = 120
NLATENT = 196
DECODER_HIDDEN_SIZE = 488
DECODER_NUM_LAYERS = 3

#adopted from https://github.com/aspuru-guzik-group/chemical_vae/tree/main/models/zinc_properties

DATA_DIR = './data'
DATA_FILE_NAME = '250k_rndm_zinc_drugs_clean_3.csv'
CHAR_FILE_NAME = 'zinc.json'

#adopted from https://github.com/ShabbirK/chemical_vae_pytorch/blob/main/data/test_idx.npy
TEST_IDX_FILE_NAME = 'test_idx.npy'

ALL_LETTERS = yaml.safe_load(open(pathlib.Path(DATA_DIR, CHAR_FILE_NAME))) + ['SOS']
N_LETTERS = len(ALL_LETTERS) 

MODELS_DIR = './models'

In [3]:
ALL_LETTERS, N_LETTERS

(['7',
  '6',
  'o',
  ']',
  '3',
  's',
  '(',
  '-',
  'S',
  '/',
  'B',
  '4',
  '[',
  ')',
  '#',
  'I',
  'l',
  'O',
  'H',
  'c',
  '1',
  '@',
  '=',
  'n',
  'P',
  '8',
  'C',
  '2',
  'F',
  '5',
  'r',
  'N',
  '+',
  '\\',
  ' ',
  'SOS'],
 36)

## Create a Utility Class

In [4]:
class VAEUtils:
    """
    This purpose of this class is 
    to help with various aspects of data processing
    """
    def __init__(self, data_dir=DATA_DIR,
                 data_file_name=DATA_FILE_NAME,
                 test_idx_file_name=TEST_IDX_FILE_NAME,
                 max_len=MAX_LEN,
                 all_letters=ALL_LETTERS):
        
        self.data_dir = pathlib.Path(data_dir)
        self.data_file = self.data_dir / pathlib.Path(data_file_name)
        self.test_file = self.data_dir / pathlib.Path(test_idx_file_name)

        self.max_len = max_len

        self.all_letters = all_letters #SMILES文字列に含まれるすべての文字
        self.n_letters = len(all_letters)
        self.letters_to_indices_dict = dict((l, i) for i, l in enumerate(all_letters)) #文字→番号
        self.indices_to_letters_dict = dict((i, l) for i, l in enumerate(all_letters)) #番号→文字

    def get_data_df(self):
        df = pd.read_csv(self.data_file) #データの読み込み
        df = df[df.smiles.str.len() <= self.max_len].reset_index(drop=True) #max_lenより長いSMILESの削除

        #preprocess input smiles to remove the newline character and add padding
        df.loc[:, 'smiles'] = df.loc[:, 'smiles'].str.strip().str.pad(width=self.max_len, side='right', fillchar=" ")

        return df

    def get_input_tensor(self, smile):
        tensor = torch.zeros(1, len(smile), self.n_letters) #batch_size * seq_length * num_features
        for i, letter in enumerate(smile):
            tensor[0][i][self.letters_to_indices_dict[letter]] = 1 #one hot vector
        return tensor

    def get_target_tensor(self, smile):
        letter_indexes = [self.letters_to_indices_dict[l] for l in smile]
        # letter_indexes.append(self.n_letters-1) #EOS
        return torch.LongTensor(letter_indexes)
    
    def get_train_valid_test_splits(self, reg_col, valid_pct=1):
        df = self.get_data_df()[['smiles', reg_col]]
        df = df.rename(columns={reg_col: 'reg_col'})

        test_idx = np.load(self.test_file)
        non_test_idx = np.array(df[~df.index.isin(test_idx)].index)
        train_idx, valid_idx = train_test_split(non_test_idx, test_size=valid_pct,
                                                random_state=42, shuffle=True)
        
        assert len(df)==len(test_idx) + len(train_idx) + len(valid_idx)

        return df, train_idx, valid_idx, test_idx
    
    def get_dl(self, df, idx, bs, shuffle=False):
        df = df.iloc[idx]

        input_tensors = torch.zeros(len(df), self.max_len, self.n_letters)
        target_tensors = torch.zeros(len(df), self.max_len, dtype=torch.long)

        for i, smile in enumerate(tqdm(df.smiles)):
            input_tensors[i] = self.get_input_tensor(smile)
            target_tensors[i] = self.get_target_tensor(smile)

        input_tensors = input_tensors
        target_tensors = target_tensors

        property_values = torch.tensor(df.reg_col.to_numpy()).type(torch.float32)

        ds = TensorDataset(input_tensors, target_tensors, property_values)
        dl = DataLoader(ds, shuffle=shuffle, batch_size=bs)

        return dl

In [5]:
vae_utils = VAEUtils()

## Model Networks

In [6]:
class Lambda(nn.Module):
    """
    This class simplifies layers from custom functions
    """
    def __init__(self, func):
        super().__init__()
        self.func = func
    
    def forward(self, x):
        return self.func(x)

In [7]:
class Encoder(nn.Module):
    
    def __init__(self, n_letters, nlatent, decoder_hidden_size):
        super().__init__()

        self.n_letters = n_letters
        self.nlatent = nlatent
        self.decoder_hidden_size = decoder_hidden_size

        self.encoder = nn.Sequential(
            Lambda(lambda x: x.permute(0, 2, 1)), # the features are in the channels dimension
            nn.Conv1d(in_channels=n_letters, out_channels=9, kernel_size=9),
            # nn.Tanh(), # the authors of the paper used tanh
            nn.ReLU(),
            nn.BatchNorm1d(9), # the authors of the paper did batch normalization
            nn.Conv1d(in_channels=9, out_channels=9, kernel_size=9),
            # nn.Tanh(), # the authors of the paper used tanh
            nn.ReLU(),
            nn.BatchNorm1d(9), # the authors of the paper did batch normalization
            nn.Conv1d(in_channels=9, out_channels=11, kernel_size=10),
            # nn.Tanh(), # the authors of the paper used tanh
            nn.ReLU(),
            nn.BatchNorm1d(11), # the authors of the paper did batch normalization
            nn.Flatten()
        )

        self.mean = nn.Linear(1045, nlatent)
        self.log_var = nn.Linear(1045, nlatent)
        self.dec_init_hidden = nn.Linear(nlatent, self.decoder_hidden_size)

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        sample = mean + (eps * std)
        return sample
    
    def forward(self, x):
        x = self.encoder(x)
        mean = self.mean(x)
        log_var = self.log_var(x)
        z = self.reparameterize(mean, log_var)
        dec_init_hidden = self.dec_init_hidden(z)

        return z,mean,log_var,dec_init_hidden

In [8]:
def preprocess_decoder_input(input_tensors):
    """
    Adjust for SOS and make batch the second demension
    """
    sos_tensor = torch.zeros(1, N_LETTERS)
    sos_tensor[0][vae_utils.letters_to_indices_dict['SOS']]
    new_tensor = torch.zeros(input_tensors.shape[0], MAX_LEN, N_LETTERS)
    new_tensor[:][0] = sos_tensor
    new_tensor[:, 1:MAX_LEN, :] = input_tensors[:, 0:MAX_LEN-1, :]
    new_tensor = new_tensor.permute(1, 0, 2).to(input_tensors.device)

    return new_tensor

In [9]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.preprocess = Lambda(preprocess_decoder_input)
        self.rnn = nn.GRU(input_size, hidden_size, num_layers)
        self.out = nn.Linear(hidden_size, input_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output, hidden = self.rnn(self.preprocess(input),)
        output = output.permute(1,0,2)
        output = self.softmax(self.out(output))
        return output, hidden

In [10]:
class PropertyPredictor(nn.Module):
    def __init__(self, nlatent):
        super().__init__()
        self.predictor = nn.Sequential(
            nn.Linear(nlatent, 1000),
            # nn.Tanh() #the authors of the paper used tanh
            nn.ReLU(),
            nn.Dropout(.2),
            nn.Linear(1000,1000),
            # nn.Tanh() #the authors of the paper used tanh
            nn.ReLU(),
            nn.Dropout(.2),
            nn.Linear(1000,1)
        )

    def forward(self, x):
        return self.predictor(x)

## Write Helper Classes & Functions for Training and Testing

In [11]:
class Trainer:
    def __init__(self, reg_col):
        self.reg_col = reg_col
        models_dir = pathlib.Path(MODELS_DIR)
        models_dir.mkdir(exist_ok=True, parents=True)

        #encoderとdecoderのモデルを保存する
        self.encoder_file = models_dir / pathlib.Path(f'{reg_col}_encoder.pth')
        self.decoder_file = models_dir / pathlib.Path(f'{reg_col}_decoder.pth')
        self.property_predictor_file = models_dir / pathlib.Path(f'{reg_col}_property_predictor.pth')

        #lossを記録する
        self.train_losses_file = models_dir / pathlib.Path(f'{reg_col}_train_losses.csv')
        self.val_losses_file = models_dir / pathlib.Path(f'{reg_col}_valid_losses.csv')

        self.loss_columns = ['total_loss', 'reconstruction_loss', 'kl_divergence', 'regression_loss']
        self.train_losses_df = pd.DataFrame(columns=self.loss_columns)
        self.val_losses_df = pd.DataFrame(columns=self.loss_columns)

    #initialize  networks
    def initialize_networks(self):
        self.encoder = Encoder(vae_utils.n_letters, NLATENT, DECODER_HIDDEN_SIZE)
        self.decoder = Decoder(vae_utils.n_letters, DECODER_HIDDEN_SIZE, DECODER_NUM_LAYERS)
        self.property_predictor = PropertyPredictor(NLATENT)

    #時間を測る
    def __time_since(self, since):
        now = time()
        s = now - since
        m = math.floor(s / 60)
        s -= m*60
        return f'{m}m {s:.0f}s'
    
    #for calculate kl divergence
    def kl_anneal_function(self, epoch, anneal_start, k=1):
        return 1/(1+np.exp(-k*(epoch - anneal_start)))
    
    def get_losses(self, epoch, device, anneal_start, input_tensors, target_tensors, property_values):
        
        # loss function 
        reconstruction_loss_func = nn.NLLLoss()
        reg_loss_func = nn.MSELoss()

        input_tensors = input_tensors.to(device)
        target_tensors = target_tensors.to(device)
        property_values = property_values.to(device)

        # encode input tensor
        z, mean, log_var, dec_init_hidden = self.encoder(input_tensors)

        # decode input tensor
        output, hidden = self.decoder(input_tensors,
                                    dec_init_hidden.unsqueeze(0).repeat(DECODER_NUM_LAYERS, 1, 1))
        
        # property predict
        reg_pred = self.property_predictor(mean)

        kl_divergence = -0.5 * torch.sum(1+log_var-mean.pow(2)-log_var.exp())
        kl_weight = self.kl_anneal_function(epoch, anneal_start)

        reg_loss = reg_loss_func(reg_pred.flatten(), property_values)
        reg_loss = reg_loss.type(torch.float32)

        reconstruction_loss = reconstruction_loss_func(output.permute(0,2,1), target_tensors)

        loss = reconstruction_loss + kl_divergence*kl_weight + reg_loss

        return loss, reconstruction_loss, kl_divergence, reg_loss

    # process of dataloader
    def process_dl(self, epoch, anneal_start, device, dl, train,
                   enc_opt=None, dec_opt=None, pp_opt=None):
        num_loaders = len(dl)
        loader_loss = 0
        recon_loss_epoch = 0
        kld_epoch = 0
        reg_loss_epoch = 0

        for input_tensors, target_tensors, property_values in dl:
            loss, reconstruction_loss, kl_divergence, reg_loss = \
                self.get_losses(epoch, device, anneal_start, input_tensors, target_tensors, property_values)
            
            if train:
                enc_opt.zero_grad()
                dec_opt.zero_grad()
                pp_opt.zero_grad()

                loss.backward()

                enc_opt.step()
                dec_opt.step()
                pp_opt.step()

            loader_loss += loss.item()
            recon_loss_epoch += reconstruction_loss.item()
            kld_epoch += kl_divergence.item()
            reg_loss_epoch += reg_loss.item()

        loader_loss /= num_loaders
        recon_loss_epoch /= num_loaders
        kld_epoch /= num_loaders
        reg_loss_epoch /= num_loaders

        return (loader_loss, recon_loss_epoch, kld_epoch, reg_loss_epoch)
    
    def save_parameters_and_losses(self,train_losses:list, val_losses:list):

        # save the models
        torch.save(self.encoder.state_dict(), self.encoder_file)
        torch.save(self.decoder.state_dict(), self.decoder_file)
        torch.save(self.property_predictor.state_dict(), self.property_predictor_file)

        # save train losses
        temp_df = pd.DataFrame(data=train_losses, columns=self.loss_columns)
        self.train_losses_df = self.train_losses_df.append(temp_df, ignore_index=True)
        self.train_losses_df.to_csv(self.train_losses_file, index=False)

        #save val losses
        temp_df = pd.DataFrame(data=val_losses, columns=self.loss_columns)
        self.val_losses_df = self.val_losses_df.append(temp_df, ignore_index=True)
        self.val_losses_df.to_csv(self.val_losses_file, index=False)

    def load_parameters_and_losses(self):

        #load the models
        self.encoder.load_state_dict(torch.load(self.decoder_file))
        self.decoder.load_state_dict(torch.load(self.decoder_file))
        self.property_predictor.load_state_dict(torch.load(self.property_predictor_file))

        #load the losses
        self.train_losses_df = pd.read_csv(self.train_losses_file)
        self.val_losses_df = pd.read_csv(self.val_losses_file)

    def print_result(self, epoch, start_time, train_loss, val_loss):
        print(f"Epoch: {epoch + 1:3d} | " + \
                  f"Train Loss: {train_loss[0]:10.5f} | Val Loss: {val_loss[0]:10.5f} | " + \
                  f"Time Taken: {self.__time_since(start_time)}")
        print(f"Train Recon Loss: {train_loss[1]:10.5f} | Val Recon Loss: {val_loss[1]:10.5f}")
        print(f"Train KLD: {train_loss[2]:10.5f} | Val KLD: {val_loss[2]:10.5f}")
        print(f"Train Reg Loss: {train_loss[3]:10.5f} | Val Reg Loss: {val_loss[3]:10.5f}\n")

    # 本体, 学習時に呼び出す関数
    def fit(self, epochs, save_every, anneal_start, lr, train_dl, valid_dl, device, load_previous=False):

        # initialize networks
        self.initialize_networks()

        # put networks on the device
        self.encoder = self.encoder.to(device)
        self.decoder = self.decoder.to(device)
        self.property_predictor = self.property_predictor.to(device)

        # 学習済みモデルをloadするとき
        if load_previous:
            self.load_parameters_and_losses()

        # optimizerの初期化
        enc_opt = optim.Adam(self.encoder.parameters(), lr=lr)
        dec_opt = optim.Adam(self.decoder.parameters(), lr=lr)
        pp_opt = optim.Adam(self.property_predictor.parameters(), lr=lr)

        # loss listの初期化
        train_losses = []
        val_losses = []
        prev_n_epochs = len(self.train_losses_df)

        # 時間を測る
        start_time = time()

        for epoch in tqdm(range(prev_n_epochs, epochs)):

            #学習モード
            self.encoder.train()
            self.decoder.train()
            self.property_predictor.train()

            #損失の計算 & 誤差伝播
            train_loss = self.process_dl(epoch, anneal_start, device, train_dl, True, enc_opt, dec_opt, pp_opt)
            train_losses.append(train_loss)

            #評価モード
            self.encoder.eval()
            self.decoder.eval()
            self.property_predictor.eval()

            with torch.no_grad():
                #損失の計算 & 誤差伝播
                val_loss = self.process_dl(epoch, anneal_start, device, valid_dl, False)
                val_losses.append(val_loss)

            # 結果の出力
            if epoch % 10 == 9:
                self.print_result(self, epoch, start_time, train_loss, val_loss)

            # モデルとlossesの保存
            if epoch % save_every == save_every - 1:
                self.save_parameters_and_losses(train_losses,val_losses)
                train_losses = []
                val_losses = []

        self.save_parameters_and_losses(train_losses, val_losses)

        del self.encoder
        del self.decoder
        del self.property_predictor

    # loss plot
    def plot_losses(self):
        
        train_losses_df = pd.read_csv(self.train_losses_file)
        val_losses_df = pd.read_csv(self.val_losses_file)
        
        plt.rcParams.update({'font.size': 15})
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,10))
        
                
        ax1.plot(train_losses_df['total_loss'], label='Train')
        ax1.plot(val_losses_df['total_loss'], label='Validation')
        ax1.set_ylabel('Total Loss')
        ax1.legend(loc='upper right')
        
        ax2.plot(train_losses_df['reconstruction_loss'], label='Train')
        ax2.plot(val_losses_df['reconstruction_loss'], label='Validation')
        ax2.set_ylabel('Reconstruction Loss')
        ax2.legend(loc='upper right')
        
        ax3.plot(train_losses_df['kl_divergence'], label='Train')
        ax3.plot(val_losses_df['kl_divergence'], label='Validation')
        ax3.set_yscale('log')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('KL Divergence')
        ax3.legend(loc='upper right')
        
        
        ax4.plot(train_losses_df['regression_loss'], label='Train')
        ax4.plot(val_losses_df['regression_loss'], label='Validation')
        ax4.set_xlabel('Epoch')
        ax4.set_ylabel('Regression Loss')
        ax4.legend(loc='upper right')
        
        plt.savefig(pathlib.Path(MODELS_DIR, f"{self.reg_col}_losses.png"))
        plt.show()

        


In [12]:
def get_dataloaders(df, train_idx, valid_idx, max_samples, bs):
    start_time = time()
    train_dl = vae_utils.get_dl(df, train_idx[:max_samples], bs, shuffle=True)
    val_dl = vae_utils.get_dl(df, valid_idx[:max_samples],bs)
    print(f'Time taken to get dataloaders: {time() - start_time:.2f}s')

    return train_dl, val_dl

In [13]:
def get_pred_mae(reg_col, df, test_idx, bs):
    encoder = Encoder(vae_utils.n_letters, NLATENT, DECODER_HIDDEN_SIZE)
    property_predictor = PropertyPredictor(NLATENT)

    encoder.load_state_dict(torch.load(pathlib.Path(MODELS_DIR, f'{reg_col}_encoder.pth')))
    property_predictor.load_state_dict(torch.load(pathlib.Path(MODELS_DIR, f'{reg_col}_property_predictor.pth')))

    dl = vae_utils.get_dl(df,test_idx, bs)

    encoder.eval()
    property_predictor.eval()

    with torch.no_grad():
        abs_errors = []
        for input_tensors, target_tensors, property_values in dl:
            z, mean, log_var, dec_init_hidden = encoder(input_tensors)
            reg_pred = property_predictor(mean)
            abs_error = torch.abs(property_values - reg_pred.flatten())
            abs_errors.append(abs_error)
        
        abs_errors = torch.cat(abs_errors)
        mae_error = abs_errors.mean()

    return mae_error

## Define Parameters for Training

In [14]:
valid_pct = .1
bs = 128
max_samples = 250000
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
epochs = 120
save_every = 2
anneal_start = 29
lr = 0.0005
load_previous = True

## Train and Test the Network for logP

In [15]:
errors = []

In [16]:
reg_col = 'logP'
df, train_idx, valid_idx, test_idx = vae_utils.get_train_valid_test_splits(reg_col, valid_pct)
trainer = Trainer(reg_col)

In [17]:
train_dl, valid_dl = get_dataloaders(df, train_idx, valid_idx, max_samples, bs)
trainer.fit(epochs, save_every, anneal_start, lr, train_dl, valid_dl, device, load_previous=False)

100%|██████████| 24891/24891 [00:18<00:00, 1328.45it/s]


Time taken to get dataloaders: 188.35s


  0%|          | 0/120 [02:42<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
trainer.plot_losses()